# 📊 Fantasy Premier League (FPL) - Complete Data Analysis & Strategy Tools

## 🎯 **Overview**
This notebook provides comprehensive analysis tools for Fantasy Premier League decision-making, including:
- **Data Exploration & Cleaning** - Understanding the dataset structure
- **Season Performance Analysis** - Player and team cumulative statistics  
- **Strategic Analysis Tools** - Fixture difficulty, player rankings, team strength
- **Actionable FPL Insights** - Real-world applications for transfers and team selection

## 📋 **Table of Contents**
1. [**Data Loading & Overview**](#data-loading--overview)
2. [**Data Cleaning & Processing**](#data-cleaning--processing)  
3. [**Exploratory Data Analysis**](#exploratory-data-analysis)
4. [**Season Statistics Calculation**](#season-statistics-calculation)
5. [**Player Performance Analysis**](#player-performance-analysis)
6. [**Strategic Analysis Tools**](#strategic-analysis-tools)
7. [**Fixture Analysis System**](#fixture-analysis-system)
8. [**Quick Reference & Usage Guide**](#quick-reference--usage-guide)

---

In [182]:
import pandas as pd 
df = pd.read_csv('fpl-data-stats.csv')
df.describe()

,id,element_type,now_cost,selected_by_percent,gameweek,minutes,shots,SoT,SiB,xG,...,defensive_contribution,xGI,npxGI,xP,total_points,PvsxP,touches,penalty_area_touches,carries_final_third,carries_penalty_area
count,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,...,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,1736.000000,1736.000000,4119.000000,4119.000000
mean,362.228211,2.546249,5.001238,2.078271,3.420005,27.210731,0.321437,0.100995,0.217043,0.034596,...,2.063122,0.059311,0.056252,1.242785,1.260257,0.017472,37.955645,1.475806,0.308813,0.122603
std,206.709011,0.834966,1.103065,6.122823,1.650746,37.810727,0.805882,0.367442,0.631449,0.128918,...,3.613342,0.172587,0.161943,2.047335,2.426631,1.420566,24.717978,1.899641,0.822402,0.526861
min,1.000000,1.000000,3.900000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-3.000000,-11.400000,0.000000,0.000000,0.000000,0.000000
25%,183.000000,2.000000,4.400000,0.100000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000
50%,364.000000,3.000000,4.800000,0.200000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,1.000000,0.000000,0.000000
75%,541.000000,3.000000,5.400000,1.000000,5.000000,70.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,0.000000,0.000000,2.083000,1.000000,0.000000,54.000000,2.000000,0.000000,0.000000
max,742.000000,4.000000,14.500000,67.400000,6.000000,90.000000,7.000000,4.000000,7.000000,2.000000,...,23.000000,2.000000,2.000000,13.000000,24.000000,12.826000,129.000000,18.000000,8.000000,11.000000


# 1️⃣ Data Loading & Overview {#data-loading--overview}

## 📂 Import Data and Initial Exploration
This section loads the FPL dataset and provides basic information about its structure.

In [183]:
# Dataset Overview and Structure
print("=== DATASET OVERVIEW ===")
print(f"Dataset Shape: {df.shape}")
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print("\n=== COLUMN NAMES ===")
print(df.columns.tolist())

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== BASIC INFO ===")
df.info()

=== DATASET OVERVIEW ===
Dataset Shape: (4119, 37)
Total Records: 4,119
Total Features: 37

=== COLUMN NAMES ===
['id', 'element_type', 'web_name', 'team_name', 'opponent_team_name', 'was_home', 'now_cost', 'selected_by_percent', 'gameweek', 'minutes', 'shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'key_passes', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'clearances_blocks_interceptions', 'recoveries', 'tackles', 'defensive_contribution', 'xGI', 'npxGI', 'xP', 'total_points', 'PvsxP', 'touches', 'penalty_area_touches', 'carries_final_third', 'carries_penalty_area']

=== DATA TYPES ===
id                                   int64
element_type                         int64
web_name                            object
team_name                           object
opponent_team_name                  object
was_home                              bool
now_cost                           float64
selected_by_percent                float64
gameweek                             int64
minutes                  

In [184]:
# Missing Values Analysis
print("=== MISSING VALUES ANALYSIS ===")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
}).sort_values('Missing Count', ascending=False)

# Display only columns with missing values
if missing_df['Missing Count'].sum() > 0:
    print(missing_df[missing_df['Missing Count'] > 0])
else:
    print("No missing values found in the dataset!")

print(f"\nTotal missing values in dataset: {missing_values.sum():,}")
print(f"Percentage of complete records: {((len(df) - missing_values.sum()) / len(df)) * 100:.2f}%")

df = df.drop(columns=['penalty_area_touches', 'touches'])

=== MISSING VALUES ANALYSIS ===
                  Column  Missing Count  Missing Percentage
34  penalty_area_touches           2383           57.853848
33               touches           2383           57.853848

Total missing values in dataset: 4,766
Percentage of complete records: -15.71%


# 2️⃣ Data Cleaning & Processing {#data-cleaning--processing}

## 🧹 Data Quality Assessment and Cleaning
Analyzing missing values, data types, and performing necessary data cleaning operations.

In [185]:
# Separate Numerical and Categorical Variables
import numpy as np

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("=== VARIABLE TYPES ===")
print(f"Numerical variables ({len(numerical_cols)}): {numerical_cols}")
print(f"\nCategorical variables ({len(categorical_cols)}): {categorical_cols}")

# For categorical variables, show unique values
print("\n=== CATEGORICAL VARIABLES ANALYSIS ===")
for col in categorical_cols[:10]:  # Show first 10 categorical columns
    unique_count = df[col].nunique()
    print(f"\n{col}:")
    print(f"  - Unique values: {unique_count}")
    if unique_count <= 20:  # Show values if not too many
        print(f"  - Values: {sorted(df[col].unique())}")
    else:
        print(f"  - Top 10 values: {df[col].value_counts().head(10).index.tolist()}")

=== VARIABLE TYPES ===
Numerical variables (31): ['id', 'element_type', 'now_cost', 'selected_by_percent', 'gameweek', 'minutes', 'shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'key_passes', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'clearances_blocks_interceptions', 'recoveries', 'tackles', 'defensive_contribution', 'xGI', 'npxGI', 'xP', 'total_points', 'PvsxP', 'carries_final_third', 'carries_penalty_area']

Categorical variables (3): ['web_name', 'team_name', 'opponent_team_name']

=== CATEGORICAL VARIABLES ANALYSIS ===

web_name:
  - Unique values: 721
  - Top 10 values: ['Patterson', 'Gomez', 'Neto', 'Anderson', 'Henderson', 'Roberts', 'James', 'Harrison', 'Phillips', 'Johnson']

team_name:
  - Unique values: 20
  - Values: ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds', 'Liverpool', 'Man City', 'Man Utd', 'Newcastle', "Nott'm Forest", 'Spurs', 'Sunderland', 'West Ham', 'Wolves']

opponent_team_n

In [186]:
# Filter useful numerical variables for FPL analysis
print("=== FILTERING USEFUL NUMERICAL VARIABLES ===")

# Define categories of useful variables
core_performance = ['total_points', 'minutes', 'now_cost', 'selected_by_percent']
attacking_metrics = ['G', 'A', 'xG', 'xA', 'shots', 'SoT', 'key_passes']
expected_metrics = ['xG', 'xA', 'xGI', 'npxG', 'npxGI', 'xP']
defensive_metrics = ['CS', 'xCS', 'GC', 'xGC', 'tackles', 'recoveries', 
                    'clearances_blocks_interceptions', 'defensive_contribution']
advanced_metrics = ['PvsxP', 'carries_final_third', 'carries_penalty_area']

# Combine into useful variables list
useful_numerical_vars = list(set(core_performance + attacking_metrics + 
                                expected_metrics + defensive_metrics + advanced_metrics))

# Filter only variables that exist in the dataset
useful_vars_available = [var for var in useful_numerical_vars if var in numerical_cols]

print(f"Original numerical variables: {len(numerical_cols)}")
print(f"Useful numerical variables: {len(useful_vars_available)}")
print(f"Variables removed: {len(numerical_cols) - len(useful_vars_available)}")

print(f"\n=== USEFUL VARIABLES BY CATEGORY ===")
print(f"Core Performance: {[v for v in core_performance if v in useful_vars_available]}")
print(f"Attacking Metrics: {[v for v in attacking_metrics if v in useful_vars_available]}")
print(f"Expected Stats: {[v for v in expected_metrics if v in useful_vars_available]}")
print(f"Defensive Metrics: {[v for v in defensive_metrics if v in useful_vars_available]}")
print(f"Advanced Metrics: {[v for v in advanced_metrics if v in useful_vars_available]}")

# Variables to exclude (less useful for FPL analysis)
excluded_vars = [var for var in numerical_cols if var not in useful_vars_available]
print(f"\n=== EXCLUDED VARIABLES ===")
print(f"Less useful for FPL: {excluded_vars}")

# Create filtered dataset with useful variables only
useful_numerical_df = df[useful_vars_available].copy()
print(f"\n=== FILTERED DATASET INFO ===")
print(f"Shape: {useful_numerical_df.shape}")
print(f"Useful numerical variables: {useful_vars_available}")

=== FILTERING USEFUL NUMERICAL VARIABLES ===
Original numerical variables: 31
Useful numerical variables: 26
Variables removed: 5

=== USEFUL VARIABLES BY CATEGORY ===
Core Performance: ['total_points', 'minutes', 'now_cost', 'selected_by_percent']
Attacking Metrics: ['G', 'A', 'xG', 'xA', 'shots', 'SoT', 'key_passes']
Expected Stats: ['xG', 'xA', 'xGI', 'npxG', 'npxGI', 'xP']
Defensive Metrics: ['CS', 'xCS', 'GC', 'xGC', 'tackles', 'recoveries', 'clearances_blocks_interceptions', 'defensive_contribution']
Advanced Metrics: ['PvsxP', 'carries_final_third', 'carries_penalty_area']

=== EXCLUDED VARIABLES ===
Less useful for FPL: ['id', 'element_type', 'gameweek', 'SiB', 'npG']

=== FILTERED DATASET INFO ===
Shape: (4119, 26)
Useful numerical variables: ['minutes', 'defensive_contribution', 'SoT', 'xCS', 'PvsxP', 'selected_by_percent', 'carries_penalty_area', 'xGI', 'clearances_blocks_interceptions', 'npxGI', 'xA', 'carries_final_third', 'total_points', 'tackles', 'GC', 'shots', 'A', 'G'

In [187]:
# Display the first 20 rows of the dataset
print("=== TOP 20 ROWS OF DATASET ===")
print(df.head(20))



=== TOP 20 ROWS OF DATASET ===
    id  element_type      web_name team_name opponent_team_name  was_home  \
0    1             1          Raya   Arsenal            Man Utd     False   
1    2             1  Arrizabalaga   Arsenal            Man Utd     False   
2    3             1          Hein   Arsenal            Man Utd     False   
3    4             1       Setford   Arsenal            Man Utd     False   
4    5             2       Gabriel   Arsenal            Man Utd     False   
5    6             2        Saliba   Arsenal            Man Utd     False   
6    7             2     Calafiori   Arsenal            Man Utd     False   
7    8             2      J.Timber   Arsenal            Man Utd     False   
8    9             2        Kiwior   Arsenal            Man Utd     False   
9   10             2  Lewis-Skelly   Arsenal            Man Utd     False   
10  11             2         White   Arsenal            Man Utd     False   
11  12             2     Zinchenko   Arsenal 

In [188]:
# Outlier Detection and Analysis
print("=== OUTLIER DETECTION ===")

def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyze outliers for key metrics
key_metrics = ['total_points', 'now_cost', 'selected_by_percent', 'minutes']

for metric in key_metrics:
    if metric in df.columns and df[metric].notna().sum() > 0:
        outliers, lower, upper = detect_outliers_iqr(df, metric)
        print(f"\n{metric.upper()}:")
        print(f"  Normal range: {lower:.2f} to {upper:.2f}")
        print(f"  Number of outliers: {len(outliers)}")
        print(f"  Percentage of outliers: {(len(outliers) / len(df)) * 100:.2f}%")
        
        if len(outliers) > 0 and len(outliers) <= 10:
            print("  Top outliers:")
            top_outliers = outliers.nlargest(10, metric)[['web_name', 'team_name', metric]]
            for _, player in top_outliers.iterrows():
                print(f"    {player['web_name']} ({player['team_name']}): {player[metric]}")

# Performance vs Expected Analysis
print("\n\n=== PERFORMANCE vs EXPECTED ANALYSIS ===")

# Players overperforming xG
if 'xG' in df.columns and 'G' in df.columns:
    df['goal_overperformance'] = df['G'] - df['xG']
    top_goal_overperformers = df[df['goal_overperformance'] > 0].nlargest(10, 'goal_overperformance')
    print("\nTop Goal Overperformers:")
    for _, player in top_goal_overperformers.iterrows():
        print(f"  {player['web_name']} ({player['team_name']}): {player['G']:.1f} goals vs {player['xG']:.2f} xG (+{player['goal_overperformance']:.2f})")

# Players overperforming xA
if 'xA' in df.columns and 'A' in df.columns:
    df['assist_overperformance'] = df['A'] - df['xA']
    top_assist_overperformers = df[df['assist_overperformance'] > 0].nlargest(10, 'assist_overperformance')
    print("\nTop Assist Overperformers:")
    for _, player in top_assist_overperformers.iterrows():
        print(f"  {player['web_name']} ({player['team_name']}): {player['A']:.1f} assists vs {player['xA']:.2f} xA (+{player['assist_overperformance']:.2f})")

=== OUTLIER DETECTION ===

TOTAL_POINTS:
  Normal range: -1.50 to 2.50
  Number of outliers: 655
  Percentage of outliers: 15.90%

NOW_COST:
  Normal range: 2.90 to 6.90
  Number of outliers: 220
  Percentage of outliers: 5.34%

SELECTED_BY_PERCENT:
  Normal range: -1.25 to 2.35
  Number of outliers: 677
  Percentage of outliers: 16.44%

MINUTES:
  Normal range: -105.00 to 175.00
  Number of outliers: 0
  Percentage of outliers: 0.00%


=== PERFORMANCE vs EXPECTED ANALYSIS ===

Top Goal Overperformers:
  Zubimendi (Arsenal): 2.0 goals vs 0.20 xG (+1.80)
  Thiago (Brentford): 2.0 goals vs 0.60 xG (+1.40)
  Richarlison (Spurs): 2.0 goals vs 0.70 xG (+1.30)
  J.Timber (Arsenal): 2.0 goals vs 0.70 xG (+1.30)
  Welbeck (Brighton): 2.0 goals vs 0.80 xG (+1.20)
  Semenyo (Bournemouth): 2.0 goals vs 0.90 xG (+1.10)
  Wood (Nott'm Forest): 2.0 goals vs 1.00 xG (+1.00)
  Isidor (Sunderland): 1.0 goals vs 0.00 xG (+1.00)
  Garner (Everton): 1.0 goals vs 0.00 xG (+1.00)
  Gravenberch (Liverpool): 

# 3️⃣ Exploratory Data Analysis {#exploratory-data-analysis}

## 🔍 Deep Dive into Data Patterns
Exploring data distributions, outliers, and relationships between variables.

In [189]:
# Positional and Team Analysis
print("=== POSITIONAL ANALYSIS ===")

# Position mapping
position_map = {1: 'Goalkeeper', 2: 'Defender', 3: 'Midfielder', 4: 'Forward'}
df['position_name'] = df['element_type'].map(position_map)

# Analysis by position
position_stats = df.groupby('position_name').agg({
    'total_points': ['count', 'mean', 'median', 'max'],
    'now_cost': ['mean', 'median'],
    'minutes': ['mean'],
    'selected_by_percent': ['mean'],
    'G': ['mean'],
    'A': ['mean']
}).round(2)

print("Position Statistics:")
print(position_stats)

print("\n=== TEAM ANALYSIS ===")

# Team performance analysis
team_stats = df.groupby('team_name').agg({
    'total_points': ['count', 'sum', 'mean'],
    'now_cost': ['mean'],
    'selected_by_percent': ['mean'],
    'G': ['sum'],
    'A': ['sum'],
    'minutes': ['sum']
}).round(2)

team_stats.columns = ['_'.join(col) for col in team_stats.columns]
team_stats = team_stats.sort_values('total_points_sum', ascending=False)

print("\nTop 10 Teams by Total Points:")
print(team_stats.head(10)[['total_points_sum', 'total_points_mean', 'now_cost_mean']])

print("\n=== VALUE ANALYSIS BY POSITION ===")
# Calculate points per million by position
df['points_per_million'] = df['total_points'] / df['now_cost']

value_by_position = df[df['total_points'] > 0].groupby('position_name')['points_per_million'].agg([
    'count', 'mean', 'median', 'max'
]).round(2)

print(value_by_position)

=== POSITIONAL ANALYSIS ===
Position Statistics:
              total_points                  now_cost        minutes  \
                     count  mean median max     mean median    mean   
position_name                                                         
Defender              1363  1.37    0.0  24     4.49    4.4   31.15   
Forward                448  1.27    0.0  16     5.79    5.4   22.14   
Goalkeeper             477  0.85    0.0  15     4.32    4.0   21.51   
Midfielder            1831  1.28    0.0  16     5.37    5.0   27.00   

              selected_by_percent     G     A  
                             mean  mean  mean  
position_name                                  
Defender                     2.09  0.01  0.02  
Forward                      3.86  0.10  0.02  
Goalkeeper                   2.34  0.00  0.00  
Midfielder                   1.56  0.04  0.04  

=== TEAM ANALYSIS ===

Top 10 Teams by Total Points:
                total_points_sum  total_points_mean  now_cost_m

In [190]:
# CORRECTED: Top Performers and Hidden Gems Analysis using CUMULATIVE season stats
print("=== TOP PERFORMERS ANALYSIS (SEASON TOTALS) ===")

# Top scorers by cumulative season points
top_scorers = season_stats.nlargest(10, 'season_points')[['web_name', 'team_name', 'position_name', 'season_points', 'now_cost', 'selected_by_percent', 'games_played']]
print("Top 10 Point Scorers (Season Total):")
for _, player in top_scorers.iterrows():
    ppg = player['season_points'] / player['games_played'] if player['games_played'] > 0 else 0
    print(f"  {player['web_name']} ({player['team_name']}, {player['position_name']}): {player['season_points']:.0f} pts in {player['games_played']} games ({ppg:.1f} ppg), £{player['now_cost']}m, {player['selected_by_percent']}% selected")

# Best value players (min 20 season points to filter out bench players)
print(f"\n=== BEST VALUE PLAYERS (Min 20 season points) ===")
value_players = season_stats[(season_stats['season_points'] >= 20) & (season_stats['points_per_million'] > 0)].nlargest(10, 'points_per_million')
print("Top 10 Value Players (Points per £m):")
for _, player in value_players.iterrows():
    print(f"  {player['web_name']} ({player['team_name']}, {player['position_name']}): {player['points_per_million']:.2f} pts/£m ({player['season_points']:.0f} pts in {player['games_played']} games, £{player['now_cost']}m)")

# Hidden gems analysis - players with strong underlying metrics but moderate total points
print(f"\n=== HIDDEN GEMS ANALYSIS (Season Stats) ===")

# Players with decent season points (30-60) but low ownership - potential for more points
hidden_gems = season_stats[(season_stats['season_points'] >= 30) & (season_stats['season_points'] <= 60) & 
                          (season_stats['selected_by_percent'] < 5) & (season_stats['selected_by_percent'] > 0) &
                          (season_stats['games_played'] >= 3)]  # Must have played at least 3 games

if len(hidden_gems) > 0:
    # Calculate underlying performance score based on expected stats
    hidden_gems = hidden_gems.copy()
    hidden_gems['underlying_score'] = (
        hidden_gems['season_xG'] * 0.3 + 
        hidden_gems['season_xA'] * 0.25 + 
        hidden_gems['season_xCS'] * 0.2 + 
        hidden_gems['season_key_passes'] * 0.1 + 
        hidden_gems['season_shots'] * 0.05 +
        (hidden_gems['season_minutes'] / (hidden_gems['games_played'] * 90)) * 0.1  # Minutes played per game ratio
    )
    
    hidden_gems_sorted = hidden_gems.nlargest(10, 'underlying_score')
    print("Players with Strong Underlying Stats but Moderate Points:")
    for _, player in hidden_gems_sorted.iterrows():
        print(f"  {player['web_name']} ({player['team_name']}, {player['position_name']}): {player['season_points']:.0f} pts, {player['selected_by_percent']}% selected")
        print(f"    Underlying: xG:{player['season_xG']:.2f}, xA:{player['season_xA']:.2f}, xCS:{player['season_xCS']:.2f}, Keys:{player['season_key_passes']:.1f} in {player['games_played']} games")
else:
    print("No hidden gems found with current criteria")

# Differential picks - low ownership but decent season points
print(f"\n=== DIFFERENTIAL PICKS (Low Ownership, Season Stats) ===")
differential_picks = season_stats[(season_stats['season_points'] >= 40) & 
                                 (season_stats['selected_by_percent'] < 3) & 
                                 (season_stats['selected_by_percent'] > 0) &
                                 (season_stats['games_played'] >= 4)]  # At least 4 games played

if len(differential_picks) > 0:
    differential_sorted = differential_picks.nlargest(10, 'season_points')
    print("High Season Points, Very Low Ownership (<3%):")
    for _, player in differential_sorted.iterrows():
        ppg = player['season_points'] / player['games_played']
        print(f"  {player['web_name']} ({player['team_name']}, {player['position_name']}): {player['season_points']:.0f} pts ({ppg:.1f} ppg), {player['selected_by_percent']:.1f}% owned, £{player['now_cost']}m")
else:
    print("No differential picks found with current criteria")

# Goal/Assist leaders with season totals
print(f"\n=== SEASON ATTACKING LEADERS ===")
goal_leaders = season_stats[season_stats['season_goals'] > 0].nlargest(8, 'season_goals')
print("Top Goal Scorers (Season Total):")
for _, player in goal_leaders.iterrows():
    gpg = player['season_goals'] / player['games_played']
    print(f"  {player['web_name']} ({player['team_name']}): {player['season_goals']:.0f} goals in {player['games_played']} games ({gpg:.2f} per game)")

assist_leaders = season_stats[season_stats['season_assists'] > 0].nlargest(8, 'season_assists')
print("\nTop Assist Providers (Season Total):")
for _, player in assist_leaders.iterrows():
    apg = player['season_assists'] / player['games_played']
    print(f"  {player['web_name']} ({player['team_name']}): {player['season_assists']:.0f} assists in {player['games_played']} games ({apg:.2f} per game)")

=== TOP PERFORMERS ANALYSIS (SEASON TOTALS) ===
Top 10 Point Scorers (Season Total):
  Haaland (Man City, Forward): 62 pts in 6 games (10.3 ppg), £14.4m, 51.0% selected
  Semenyo (Bournemouth, Midfielder): 48 pts in 6 games (8.0 ppg), £7.8m, 52.1% selected
  Senesi (Bournemouth, Defender): 44 pts in 6 games (7.3 ppg), £4.8m, 18.9% selected
  Guéhi (Crystal Palace, Defender): 43 pts in 6 games (7.2 ppg), £4.8m, 26.3% selected
  Anthony (Burnley, Midfielder): 40 pts in 6 games (6.7 ppg), £5.6m, 3.6% selected
  Alderete (Sunderland, Defender): 39 pts in 6 games (6.5 ppg), £4.0m, 2.9% selected
  Enzo (Chelsea, Midfielder): 39 pts in 6 games (6.5 ppg), £6.7m, 13.0% selected
  Roefs (Sunderland, Goalkeeper): 39 pts in 6 games (6.5 ppg), £4.5m, 2.8% selected
  João Pedro (Chelsea, Forward): 37 pts in 6 games (6.2 ppg), £7.8m, 67.4% selected
  Caicedo (Chelsea, Midfielder): 35 pts in 6 games (5.8 ppg), £5.7m, 12.1% selected

=== BEST VALUE PLAYERS (Min 20 season points) ===
Top 10 Value Player

# 5️⃣ Player Performance Analysis {#player-performance-analysis}

## 🏆 Season Leaders, Value Picks & Hidden Gems
Analysis of top performers using **cumulative season statistics** (not single gameweek data).

In [191]:
# Calculate cumulative season statistics for each player
print("=== CALCULATING CUMULATIVE SEASON STATISTICS ===")

# Group by player and calculate season totals
season_stats = df.groupby(['web_name', 'team_name', 'element_type', 'now_cost', 'selected_by_percent']).agg({
    'total_points': 'sum',  # Sum of all gameweek points
    'minutes': 'sum',       # Total minutes played
    'G': 'sum',            # Total goals
    'A': 'sum',            # Total assists  
    'xG': 'sum',           # Total expected goals
    'xA': 'sum',           # Total expected assists
    'shots': 'sum',        # Total shots
    'SoT': 'sum',          # Total shots on target
    'key_passes': 'sum',   # Total key passes
    'CS': 'sum',           # Total clean sheets
    'xCS': 'sum',          # Total expected clean sheets
    'GC': 'sum',           # Total goals conceded
    'xGC': 'sum',          # Total expected goals conceded
    'gameweek': ['count', 'max'],  # Games played and latest gameweek
    'SiB': 'sum',          # Total shots in box
    'tackles': 'sum',      # Total tackles
    'recoveries': 'sum'    # Total recoveries
}).round(2)

print("Columns after aggregation:")
print(season_stats.columns.tolist())

# Flatten column names
season_stats.columns = ['_'.join(col) if col[1] else col[0] for col in season_stats.columns]
season_stats = season_stats.rename(columns={
    'gameweek_count': 'games_played',
    'gameweek_max': 'last_gameweek'
})

print("Columns after flattening:")
print(season_stats.columns.tolist())

# Reset index to make it a regular dataframe
season_stats = season_stats.reset_index()

# Add position names
position_map = {1: 'Goalkeeper', 2: 'Defender', 3: 'Midfielder', 4: 'Forward'}
season_stats['position_name'] = season_stats['element_type'].map(position_map)

# Calculate additional metrics using the correct column names
season_stats['points_per_million'] = season_stats['total_points_sum'] / season_stats['now_cost']
season_stats['points_per_game'] = season_stats['total_points_sum'] / season_stats['games_played']
season_stats['minutes_per_game'] = season_stats['minutes_sum'] / season_stats['games_played']
season_stats['goals_per_game'] = season_stats['G_sum'] / season_stats['games_played']
season_stats['assists_per_game'] = season_stats['A_sum'] / season_stats['games_played']

# Rename main columns for clarity
season_stats = season_stats.rename(columns={
    'total_points_sum': 'season_points',
    'minutes_sum': 'season_minutes',
    'G_sum': 'season_goals',
    'A_sum': 'season_assists',
    'xG_sum': 'season_xG',
    'xA_sum': 'season_xA',
    'shots_sum': 'season_shots',
    'SoT_sum': 'season_SoT',
    'key_passes_sum': 'season_key_passes',
    'CS_sum': 'season_CS',
    'xCS_sum': 'season_xCS',
    'GC_sum': 'season_GC',
    'xGC_sum': 'season_xGC',
    'SiB_sum': 'season_SiB',
    'tackles_sum': 'season_tackles',
    'recoveries_sum': 'season_recoveries'
})

# Round all numeric columns
numeric_cols = season_stats.select_dtypes(include=[np.number]).columns
season_stats[numeric_cols] = season_stats[numeric_cols].round(2)

print(f"Created season stats for {len(season_stats)} players")
print(f"Data covers gameweeks 1-{df['gameweek'].max()}")
print("\nSample of season stats:")
print(season_stats[['web_name', 'team_name', 'position_name', 'games_played', 'season_points', 'season_goals', 'season_assists', 'season_minutes']].head().to_string(index=False))

=== CALCULATING CUMULATIVE SEASON STATISTICS ===
Columns after aggregation:
[('total_points', 'sum'), ('minutes', 'sum'), ('G', 'sum'), ('A', 'sum'), ('xG', 'sum'), ('xA', 'sum'), ('shots', 'sum'), ('SoT', 'sum'), ('key_passes', 'sum'), ('CS', 'sum'), ('xCS', 'sum'), ('GC', 'sum'), ('xGC', 'sum'), ('gameweek', 'count'), ('gameweek', 'max'), ('SiB', 'sum'), ('tackles', 'sum'), ('recoveries', 'sum')]
Columns after flattening:
['total_points_sum', 'minutes_sum', 'G_sum', 'A_sum', 'xG_sum', 'xA_sum', 'shots_sum', 'SoT_sum', 'key_passes_sum', 'CS_sum', 'xCS_sum', 'GC_sum', 'xGC_sum', 'games_played', 'last_gameweek', 'SiB_sum', 'tackles_sum', 'recoveries_sum']
Created season stats for 758 players
Data covers gameweeks 1-6

Sample of season stats:
 web_name   team_name position_name  games_played  season_points  season_goals  season_assists  season_minutes
 A.Becker   Liverpool    Goalkeeper             6             20           0.0             0.0             540
 A.García Aston Villa      

# 4️⃣ Season Statistics Calculation {#season-statistics-calculation}

## ⚡ Converting Gameweek Data to Season Totals  
**🔑 KEY INSIGHT**: The original data contains gameweek-by-gameweek statistics, not season totals.  
This section aggregates all gameweek data to create proper cumulative season statistics for accurate analysis.

# 6️⃣ Strategic Analysis Tools {#strategic-analysis-tools}

## ⚔️ Advanced FPL Analysis Functions

This section contains powerful, reusable functions for Fantasy Premier League strategic analysis:

### 🔧 **Available Tools:**
1. **Defender Rankings** - Rank defenders by clean sheet potential and value
2. **Attacker Rankings** - Rank attacking players by goal/assist potential  
3. **Team Strength Analysis** - Calculate attacking and defensive strength for all teams
4. **Fixture Difficulty Calculator** - Score any specific matchup

### 📊 **Key Features:**
- Uses **cumulative season statistics** for accuracy
- Considers expected stats (xG, xA, xCS) for sustainability  
- Includes value scoring (points per £million)
- Accounts for consistency and minutes played
- Easily customizable parameters

In [192]:
import pandas as pd
import numpy as np
from typing import Optional, List

def calculate_team_stats_corrected(season_data: pd.DataFrame) -> tuple:
    """
    Calculate attacking and defensive statistics for each team using cumulative season data.
    
    Args:
        season_data: DataFrame with cumulative season statistics per player
        
    Returns:
        tuple: (attacking_stats, defensive_stats) DataFrames
    """
    # Attacking stats by team (aggregate all players from each team)
    attacking_stats = season_data.groupby('team_name').agg({
        'season_xG': 'sum',      # Total team xG
        'season_goals': 'sum',   # Total team goals
        'season_shots': 'sum',   # Total team shots
        'season_SoT': 'sum',     # Total team shots on target
        'season_minutes': 'sum', # Total team minutes
        'games_played': 'mean'   # Average games played (should be similar for all players)
    }).round(3)
    
    # Convert totals to per-game averages
    attacking_stats['avg_xG_for'] = attacking_stats['season_xG'] / attacking_stats['games_played']
    attacking_stats['avg_G_for'] = attacking_stats['season_goals'] / attacking_stats['games_played']
    attacking_stats['avg_shots_for'] = attacking_stats['season_shots'] / attacking_stats['games_played']
    attacking_stats['avg_SoT_for'] = attacking_stats['season_SoT'] / attacking_stats['games_played']
    
    # For defensive stats, we need to use the original gameweek data to get opponent information
    # Since we don't have opponent data in season_stats, we'll use a simplified approach
    # based on goals conceded for defensive teams (GK + DEF)
    defensive_players = season_data[season_data['element_type'].isin([1, 2])]  # GK and DEF
    
    defensive_stats = defensive_players.groupby('team_name').agg({
        'season_GC': 'mean',     # Average goals conceded per defensive player
        'season_xGC': 'mean',    # Average xGC per defensive player  
        'games_played': 'mean'   # Average games played
    }).round(3)
    
    # Convert to per-game averages (rename for consistency)
    defensive_stats['avg_G_conceded'] = defensive_stats['season_GC'] / defensive_stats['games_played']
    defensive_stats['avg_xG_conceded'] = defensive_stats['season_xGC'] / defensive_stats['games_played']
    
    return attacking_stats, defensive_stats

def rank_fixtures_corrected(season_data: pd.DataFrame, upcoming_gameweeks: Optional[List[int]] = None) -> pd.DataFrame:
    """
    Analyze and rank fixtures based on attacking strength vs defensive weakness using season data.
    
    Args:
        season_data: DataFrame with cumulative season statistics
        upcoming_gameweeks: List of gameweek numbers to analyze (if None, uses next 3 GWs)
    
    Returns:
        DataFrame with ranked fixtures showing favorability scores
    """
    if upcoming_gameweeks is None:
        current_gw = season_data['last_gameweek'].max()
        upcoming_gameweeks = [current_gw + 1, current_gw + 2, current_gw + 3]
    
    # Get team statistics
    attacking_stats, defensive_stats = calculate_team_stats_corrected(season_data)
    
    # Create fixtures matrix
    teams = season_data['team_name'].unique()
    fixtures = []
    
    for gw in upcoming_gameweeks:
        for home_team in teams:
            for away_team in teams:
                if home_team != away_team:
                    fixtures.append({
                        'gameweek': gw,
                        'home_team': home_team,
                        'away_team': away_team,
                        'fixture': f"{home_team} vs {away_team}"
                    })
    
    fixture_df = pd.DataFrame(fixtures)
    
    # Add attacking stats for home team
    fixture_df = fixture_df.merge(
        attacking_stats[['avg_xG_for', 'avg_G_for', 'avg_shots_for', 'avg_SoT_for']], 
        left_on='home_team', 
        right_index=True, 
        how='left'
    )
    
    # Add defensive stats for away team
    fixture_df = fixture_df.merge(
        defensive_stats[['avg_xG_conceded', 'avg_G_conceded']], 
        left_on='away_team', 
        right_index=True, 
        how='left'
    )
    
    # Calculate favorability scores
    fixture_df['attacking_strength'] = (
        fixture_df['avg_xG_for'] * 0.4 + 
        fixture_df['avg_G_for'] * 0.3 + 
        fixture_df['avg_shots_for'] * 0.2 + 
        fixture_df['avg_SoT_for'] * 0.1
    )
    
    fixture_df['defensive_weakness'] = (
        fixture_df['avg_xG_conceded'] * 0.6 + 
        fixture_df['avg_G_conceded'] * 0.4
    )
    
    # Overall favorability score
    fixture_df['favorability_score'] = (
        fixture_df['attacking_strength'] * 0.6 + 
        fixture_df['defensive_weakness'] * 0.4
    )
    
    # Add difficulty rating
    fixture_df['difficulty_rating'] = pd.cut(
        fixture_df['favorability_score'], 
        bins=5, 
        labels=['Very Hard', 'Hard', 'Medium', 'Easy', 'Very Easy']
    )
    
    # Sort by favorability
    result = fixture_df.sort_values(['gameweek', 'favorability_score'], ascending=[True, False])
    
    output_cols = [
        'gameweek', 'fixture', 'home_team', 'away_team', 'favorability_score', 
        'difficulty_rating', 'attacking_strength', 'defensive_weakness',
        'avg_xG_for', 'avg_G_for', 'avg_xG_conceded', 'avg_G_conceded'
    ]
    
    return result[output_cols].round(3)

print("=== CORRECTED FIXTURE ANALYSIS FUNCTION CREATED ===")
print("Function: rank_fixtures_corrected(season_data, upcoming_gameweeks=None)")
print("Purpose: Identifies favorable fixtures using CUMULATIVE season statistics")
print("\nKey Changes:")
print("- Uses season_stats dataframe instead of gameweek data")
print("- Calculates team attacking/defensive strength from cumulative player stats")
print("- More accurate representation of team form over the season")

=== CORRECTED FIXTURE ANALYSIS FUNCTION CREATED ===
Function: rank_fixtures_corrected(season_data, upcoming_gameweeks=None)
Purpose: Identifies favorable fixtures using CUMULATIVE season statistics

Key Changes:
- Uses season_stats dataframe instead of gameweek data
- Calculates team attacking/defensive strength from cumulative player stats
- More accurate representation of team form over the season


In [193]:
def filter_defenders_corrected(season_data: pd.DataFrame, min_games: int = 3, top_n: int = 20) -> pd.DataFrame:
    """
    Rank defenders by clean sheet potential using cumulative season data.
    
    Args:
        season_data: DataFrame with cumulative season statistics
        min_games: Minimum games played to be considered
        top_n: Number of top defenders to return
    
    Returns:
        DataFrame with ranked defenders based on season performance
    """
    # Filter for defenders only
    defenders = season_data[season_data['element_type'] == 2].copy()
    
    # Filter by minimum games played
    defenders = defenders[defenders['games_played'] >= min_games]
    
    if len(defenders) == 0:
        return pd.DataFrame()
    
    # Calculate performance metrics
    defenders['clean_sheet_rate'] = (defenders['season_CS'] / defenders['games_played']).fillna(0)
    defenders['xCS_per_game'] = (defenders['season_xCS'] / defenders['games_played']).fillna(0)
    defenders['goals_conceded_per_game'] = (defenders['season_GC'] / defenders['games_played']).fillna(0)
    defenders['minutes_per_game'] = defenders['season_minutes'] / defenders['games_played']
    defenders['consistency_score'] = np.minimum(defenders['minutes_per_game'] / 90, 1)
    
    # Clean sheet potential score
    defenders['clean_sheet_potential'] = (
        defenders['xCS_per_game'] * 0.4 +
        defenders['clean_sheet_rate'] * 0.35 +
        (1 / (defenders['goals_conceded_per_game'] + 0.1)) * 0.15 +  # Lower goals conceded = better
        defenders['consistency_score'] * 0.1
    )
    
    # Value score
    defenders['value_score'] = defenders['season_points'] / defenders['now_cost']
    
    # Overall defender score  
    defenders['defender_score'] = (
        defenders['clean_sheet_potential'] * 0.6 +
        defenders['value_score'] * 0.25 +
        defenders['consistency_score'] * 0.15
    )
    
    # Sort by defender score
    result = defenders.sort_values('defender_score', ascending=False)
    
    # Select key columns
    output_cols = [
        'web_name', 'team_name', 'now_cost', 'selected_by_percent',
        'defender_score', 'clean_sheet_potential', 'value_score', 'consistency_score',
        'games_played', 'clean_sheet_rate', 'xCS_per_game', 'goals_conceded_per_game',
        'season_points', 'season_minutes', 'season_CS', 'season_xCS'
    ]
    
    return result[output_cols].head(top_n).round(3)

def filter_attackers_corrected(season_data: pd.DataFrame, min_games: int = 3, top_n: int = 20, positions: List[int] = [3, 4]) -> pd.DataFrame:
    """
    Rank attackers using cumulative season data.
    
    Args:
        season_data: DataFrame with cumulative season statistics
        min_games: Minimum games played to be considered
        top_n: Number of top attackers to return
        positions: List of position types to include (3=Midfielder, 4=Forward)
    
    Returns:
        DataFrame with ranked attackers based on season performance
    """
    # Filter for attackers
    attackers = season_data[season_data['element_type'].isin(positions)].copy()
    
    # Filter by minimum games
    attackers = attackers[attackers['games_played'] >= min_games]
    
    if len(attackers) == 0:
        return pd.DataFrame()
    
    # Calculate performance metrics
    attackers['goals_per_game'] = (attackers['season_goals'] / attackers['games_played']).fillna(0)
    attackers['assists_per_game'] = (attackers['season_assists'] / attackers['games_played']).fillna(0)
    attackers['xG_per_game'] = (attackers['season_xG'] / attackers['games_played']).fillna(0)
    attackers['xA_per_game'] = (attackers['season_xA'] / attackers['games_played']).fillna(0)
    attackers['shots_per_game'] = (attackers['season_shots'] / attackers['games_played']).fillna(0)
    attackers['SoT_per_game'] = (attackers['season_SoT'] / attackers['games_played']).fillna(0)
    attackers['SiB_per_game'] = (attackers['season_SiB'] / attackers['games_played']).fillna(0)
    attackers['minutes_per_game'] = attackers['season_minutes'] / attackers['games_played']
    
    # Attacking threat score
    attackers['attacking_threat'] = (
        attackers['xG_per_game'] * 0.3 +
        attackers['xA_per_game'] * 0.25 +
        attackers['goals_per_game'] * 0.2 +
        attackers['assists_per_game'] * 0.15 +
        attackers['SoT_per_game'] * 0.05 +
        attackers['SiB_per_game'] * 0.05
    )
    
    # Consistency score
    attackers['consistency_score'] = np.minimum(attackers['minutes_per_game'] / 90, 1)
    
    # Value score
    attackers['value_score'] = attackers['season_points'] / attackers['now_cost']
    
    # Overall attacker score
    attackers['attacker_score'] = (
        attackers['attacking_threat'] * 0.6 +
        attackers['value_score'] * 0.25 +
        attackers['consistency_score'] * 0.15
    )
    
    # Sort by attacker score
    result = attackers.sort_values('attacker_score', ascending=False)
    
    # Select key columns
    output_cols = [
        'web_name', 'team_name', 'position_name', 'now_cost', 'selected_by_percent',
        'attacker_score', 'attacking_threat', 'value_score', 'consistency_score',
        'games_played', 'goals_per_game', 'assists_per_game', 'xG_per_game', 'xA_per_game',
        'shots_per_game', 'SoT_per_game', 'season_points', 'season_minutes'
    ]
    
    return result[output_cols].head(top_n).round(3)

print("=== CORRECTED DEFENDER & ATTACKER FILTERING FUNCTIONS CREATED ===")
print("Functions: filter_defenders_corrected() & filter_attackers_corrected()")
print("Purpose: Rank players using CUMULATIVE season statistics")
print("\nKey Changes:")
print("- Uses season_stats dataframe with cumulative data")
print("- Changed min_minutes to min_games for more intuitive filtering")
print("- Calculates per-game averages from season totals")
print("- More accurate player performance assessment")

=== CORRECTED DEFENDER & ATTACKER FILTERING FUNCTIONS CREATED ===
Functions: filter_defenders_corrected() & filter_attackers_corrected()
Purpose: Rank players using CUMULATIVE season statistics

Key Changes:
- Uses season_stats dataframe with cumulative data
- Changed min_minutes to min_games for more intuitive filtering
- Calculates per-game averages from season totals
- More accurate player performance assessment


In [194]:
# PROBLEM ANALYSIS: Why Current Fixture Analysis is Wrong
print("=== FIXTURE ANALYSIS PROBLEM IDENTIFICATION ===")
print("🚫 CURRENT ISSUE:")
print("• We're creating fake fixtures (every team vs every team)")
print("• Real FPL has specific fixtures each gameweek")
print("• We need actual fixture data to make this analysis meaningful")

print(f"\n📊 WHAT WE HAVE:")
print(f"• Historical performance data (gameweeks 1-{df['gameweek'].max()})")
print(f"• Team attacking/defensive strength from season data")
print(f"• Player performance metrics")

print(f"\n❓ WHAT WE'RE MISSING:")
print("• Actual fixtures for upcoming gameweeks (who plays whom)")
print("• Home/away venue information")
print("• Real fixture difficulty from FPL API")

print(f"\n💡 SOLUTIONS WE CAN IMPLEMENT:")
print("1. **Team Strength Rankings** - Rank teams by attack/defense for manual fixture lookup")
print("2. **Player vs Team Analysis** - Show how players perform against specific team types")
print("3. **Fixture Difficulty Scoring** - Create a system to score any matchup")
print("4. **Historical Fixture Analysis** - Analyze past fixtures for patterns")
print("5. **Mock Upcoming Analysis** - Use common upcoming fixtures as examples")

=== FIXTURE ANALYSIS PROBLEM IDENTIFICATION ===
🚫 CURRENT ISSUE:
• We're creating fake fixtures (every team vs every team)
• Real FPL has specific fixtures each gameweek
• We need actual fixture data to make this analysis meaningful

📊 WHAT WE HAVE:
• Historical performance data (gameweeks 1-6)
• Team attacking/defensive strength from season data
• Player performance metrics

❓ WHAT WE'RE MISSING:
• Actual fixtures for upcoming gameweeks (who plays whom)
• Home/away venue information
• Real fixture difficulty from FPL API

💡 SOLUTIONS WE CAN IMPLEMENT:
1. **Team Strength Rankings** - Rank teams by attack/defense for manual fixture lookup
2. **Player vs Team Analysis** - Show how players perform against specific team types
3. **Fixture Difficulty Scoring** - Create a system to score any matchup
4. **Historical Fixture Analysis** - Analyze past fixtures for patterns
5. **Mock Upcoming Analysis** - Use common upcoming fixtures as examples


# 7️⃣ Fixture Analysis System {#fixture-analysis-system}

## 🏟️ **Smart Fixture Analysis for FPL Decision Making**

### ❌ **The Problem with Traditional Fixture Analysis**
Most FPL tools create fake fixtures or rely on outdated data. Our approach is different and **much more practical**.

### ✅ **Our Solution: Real-World Applicable Tools**

In [195]:
# 🏆 SOLUTION 1: Team Strength Rankings
print("="*70)
print("📊 TEAM STRENGTH RANKINGS (Based on Season Performance)")
print("="*70)
print("💡 Use these rankings to assess fixture difficulty manually")

def create_team_strength_rankings(season_data: pd.DataFrame) -> pd.DataFrame:
    """
    Create team strength rankings based on season performance.
    Users can use this to manually assess fixture difficulty.
    """
    # Calculate team stats from player data
    attacking_stats = season_data.groupby('team_name').agg({
        'season_goals': 'sum',
        'season_xG': 'sum', 
        'season_shots': 'sum',
        'season_SoT': 'sum',
        'games_played': 'mean'
    }).round(2)
    
    defensive_stats = season_data[season_data['element_type'].isin([1, 2])].groupby('team_name').agg({
        'season_CS': 'mean',
        'season_xCS': 'mean',
        'season_GC': 'mean',
        'season_xGC': 'mean',
        'games_played': 'mean'
    }).round(2)
    
    # Convert to per-game averages
    attacking_stats['goals_per_game'] = attacking_stats['season_goals'] / attacking_stats['games_played']
    attacking_stats['xG_per_game'] = attacking_stats['season_xG'] / attacking_stats['games_played']
    attacking_stats['shots_per_game'] = attacking_stats['season_shots'] / attacking_stats['games_played']
    
    defensive_stats['CS_rate'] = defensive_stats['season_CS'] / defensive_stats['games_played']
    defensive_stats['GC_per_game'] = defensive_stats['season_GC'] / defensive_stats['games_played']
    
    # Calculate strength scores
    attacking_stats['attack_strength'] = (
        attacking_stats['xG_per_game'] * 0.4 +
        attacking_stats['goals_per_game'] * 0.3 +
        attacking_stats['shots_per_game'] * 0.3
    )
    
    defensive_stats['defense_strength'] = (
        defensive_stats['CS_rate'] * 0.4 +
        (1 / (defensive_stats['GC_per_game'] + 0.1)) * 0.6  # Lower goals conceded = stronger
    )
    
    # Combine into rankings
    team_rankings = attacking_stats[['attack_strength']].join(
        defensive_stats[['defense_strength']], how='outer'
    )
    
    team_rankings = team_rankings.fillna(0)
    team_rankings['overall_strength'] = (
        team_rankings['attack_strength'] * 0.6 + 
        team_rankings['defense_strength'] * 0.4
    )
    
    # Add rankings
    team_rankings['attack_rank'] = team_rankings['attack_strength'].rank(ascending=False, method='dense').astype(int)
    team_rankings['defense_rank'] = team_rankings['defense_strength'].rank(ascending=False, method='dense').astype(int)
    team_rankings['overall_rank'] = team_rankings['overall_strength'].rank(ascending=False, method='dense').astype(int)
    
    return team_rankings.round(3)

# Create team rankings
team_rankings = create_team_strength_rankings(season_stats)
team_rankings_sorted = team_rankings.sort_values('overall_rank')

print("🏆 TEAM STRENGTH RANKINGS (Season Performance)")
print("=" * 60)
print("Overall Team Rankings:")
print(team_rankings_sorted[['overall_rank', 'attack_rank', 'defense_rank', 'overall_strength', 'attack_strength', 'defense_strength']].head(15).to_string())

print(f"\n⚽ TOP 8 ATTACKING TEAMS:")
attack_rankings = team_rankings.sort_values('attack_rank').head(8)
for idx, (team, data) in enumerate(attack_rankings.iterrows(), 1):
    print(f"{int(data['attack_rank']):2d}. {team:<15} (Attack: {data['attack_strength']:.3f})")

print(f"\n🛡️ TOP 8 DEFENSIVE TEAMS:")
defense_rankings = team_rankings.sort_values('defense_rank').head(8)
for idx, (team, data) in enumerate(defense_rankings.iterrows(), 1):
    print(f"{int(data['defense_rank']):2d}. {team:<15} (Defense: {data['defense_strength']:.3f})")

📊 TEAM STRENGTH RANKINGS (Based on Season Performance)
💡 Use these rankings to assess fixture difficulty manually
🏆 TEAM STRENGTH RANKINGS (Season Performance)
Overall Team Rankings:
                overall_rank  attack_rank  defense_rank  overall_strength  attack_strength  defense_strength
team_name                                                                                                   
Liverpool                  1            1            11             3.885            6.067             0.612
Arsenal                    2            4             1             3.848            5.453             1.442
Man Utd                    3            2            18             3.754            6.052             0.308
Man City                   4            3             8             3.599            5.539             0.689
Crystal Palace             5            7             2             3.530            5.092             1.188
Chelsea                    6            5            1

In [196]:
# 🎯 SOLUTION 2: Fixture Difficulty Calculator
print("\n" + "="*70)
print("🧮 FIXTURE DIFFICULTY CALCULATOR - Test Any Matchup!")
print("="*70)

def calculate_fixture_difficulty(home_team: str, away_team: str, team_rankings: pd.DataFrame, 
                                attacking_player: bool = True, home_advantage: float = 0.1) -> dict:
    """
    Calculate fixture difficulty for a specific matchup.
    
    Args:
        home_team: Home team name
        away_team: Away team name  
        team_rankings: DataFrame with team strength rankings
        attacking_player: True if analyzing attacking player, False for defender/GK
        home_advantage: Home advantage factor (default 0.1)
    
    Returns:
        Dictionary with difficulty analysis
    """
    
    if home_team not in team_rankings.index or away_team not in team_rankings.index:
        return {"error": "Team not found in rankings"}
    
    home_stats = team_rankings.loc[home_team]
    away_stats = team_rankings.loc[away_team]
    
    if attacking_player:
        # For attacking players: home team attack vs away team defense
        attack_strength = home_stats['attack_strength'] * (1 + home_advantage)
        defense_strength = away_stats['defense_strength']
        favorability = attack_strength - defense_strength
        
        analysis = {
            'fixture': f"{home_team} vs {away_team}",
            'for_attacking_players': True,
            'attack_strength': attack_strength,
            'opponent_defense': defense_strength,
            'favorability_score': favorability,
            'difficulty': 'Very Easy' if favorability > 2.5 else
                         'Easy' if favorability > 1.5 else
                         'Medium' if favorability > 0.5 else
                         'Hard' if favorability > -0.5 else 'Very Hard',
            'recommendation': 'Strong Pick' if favorability > 2.0 else
                            'Good Pick' if favorability > 1.0 else
                            'Average Pick' if favorability > 0 else
                            'Avoid' if favorability > -1.0 else 'Strong Avoid'
        }
    else:
        # For defenders/GKs: home team defense vs away team attack
        defense_strength = home_stats['defense_strength'] * (1 + home_advantage)
        attack_threat = away_stats['attack_strength']
        favorability = defense_strength - attack_threat
        
        analysis = {
            'fixture': f"{home_team} vs {away_team}",
            'for_attacking_players': False,
            'defense_strength': defense_strength,
            'opponent_attack': attack_threat,
            'favorability_score': favorability,
            'difficulty': 'Very Easy' if favorability > 1.0 else
                         'Easy' if favorability > 0.5 else
                         'Medium' if favorability > 0 else
                         'Hard' if favorability > -0.5 else 'Very Hard',
            'recommendation': 'Strong Pick' if favorability > 0.8 else
                            'Good Pick' if favorability > 0.3 else
                            'Average Pick' if favorability > -0.2 else
                            'Avoid' if favorability > -0.8 else 'Strong Avoid'
        }
    
    return analysis

# Example fixture calculations
print("\n🏟️ EXAMPLE FIXTURE DIFFICULTY CALCULATIONS:")
print("=" * 50)

example_fixtures = [
    ("AFC Bournemouth", "Fulham"),
    ("Arsenal", "West Ham"),
    ("Aston Villa", "Burnley"),
    ("Brentford", "Manchester City"),
    ("Chelsea", "Liverpool"),
    ("Everton", "Crystal Palace"),
    ("Leeds United", "Tottenham Hotspur"),
    ("Manchester United", "Sunderland"),
    ("Newcastle United", "Nottingham Forest"),
    ("Wolves", "Brighton & Hove Albion")
]

for home, away in example_fixtures:
    if home in team_rankings.index and away in team_rankings.index:
        # For attacking players
        attack_analysis = calculate_fixture_difficulty(home, away, team_rankings, attacking_player=True)
        defense_analysis = calculate_fixture_difficulty(home, away, team_rankings, attacking_player=False)
        
        print(f"\n📍 {attack_analysis['fixture']}:")
        print(f"   🗡️  Attackers: {attack_analysis['difficulty']:10} | {attack_analysis['recommendation']:12} | Score: {attack_analysis['favorability_score']:+.2f}")
        print(f"   🛡️  Defenders: {defense_analysis['difficulty']:10} | {defense_analysis['recommendation']:12} | Score: {defense_analysis['favorability_score']:+.2f}")

print(f"\n💡 HOW TO USE:")
print("• Positive score = Favorable fixture")
print("• Negative score = Difficult fixture") 
print("• Use this calculator with real upcoming fixtures from FPL website")
print("• Input: calculate_fixture_difficulty('Home Team', 'Away Team', team_rankings)")


🧮 FIXTURE DIFFICULTY CALCULATOR - Test Any Matchup!

🏟️ EXAMPLE FIXTURE DIFFICULTY CALCULATIONS:

📍 Arsenal vs West Ham:
   🗡️  Attackers: Very Easy  | Strong Pick  | Score: +5.70
   🛡️  Defenders: Very Hard  | Strong Avoid | Score: -2.40

📍 Aston Villa vs Burnley:
   🗡️  Attackers: Very Easy  | Strong Pick  | Score: +3.76
   🛡️  Defenders: Very Hard  | Strong Avoid | Score: -2.39

📍 Chelsea vs Liverpool:
   🗡️  Attackers: Very Easy  | Strong Pick  | Score: +5.26
   🛡️  Defenders: Very Hard  | Strong Avoid | Score: -5.46

📍 Everton vs Crystal Palace:
   🗡️  Attackers: Very Easy  | Strong Pick  | Score: +3.81
   🛡️  Defenders: Very Hard  | Strong Avoid | Score: -4.31

💡 HOW TO USE:
• Positive score = Favorable fixture
• Negative score = Difficult fixture
• Use this calculator with real upcoming fixtures from FPL website
• Input: calculate_fixture_difficulty('Home Team', 'Away Team', team_rankings)


In [197]:
# SOLUTION 3: Player Recommendations for Team Matchups
print("\n" + "="*80)
print("=== SOLUTION 3: PLAYER RECOMMENDATIONS BY OPPONENT STRENGTH ===")
print("Find best players against weak defenses/attacks")

def get_players_for_matchup(team: str, opponent_type: str, season_data: pd.DataFrame, 
                           team_rankings: pd.DataFrame, top_n: int = 8) -> pd.DataFrame:
    """
    Get player recommendations based on opponent strength.
    
    Args:
        team: Team name to get players from
        opponent_type: 'weak_defense' for attackers, 'weak_attack' for defenders
        season_data: Player season statistics
        team_rankings: Team strength rankings
        top_n: Number of players to return
    """
    team_players = season_data[season_data['team_name'] == team].copy()
    
    if len(team_players) == 0:
        return pd.DataFrame()
    
    if opponent_type == 'weak_defense':
        # Get attacking players when facing weak defenses
        attackers = team_players[team_players['element_type'].isin([3, 4])]  # MID + FWD
        attackers = attackers[attackers['games_played'] >= 3]
        
        if len(attackers) == 0:
            return pd.DataFrame()
            
        # Score based on attacking output and value
        attackers['matchup_score'] = (
            attackers['goals_per_game'] * 3 +
            attackers['assists_per_game'] * 2 +
            attackers['points_per_game'] * 0.5 +
            attackers['points_per_million'] * 0.3
        )
        
        result = attackers.sort_values('matchup_score', ascending=False).head(top_n)
        return result[['web_name', 'position_name', 'season_points', 'now_cost', 
                      'goals_per_game', 'assists_per_game', 'points_per_game', 
                      'points_per_million', 'matchup_score']].round(2)
        
    else:  # weak_attack
        # Get defensive players when facing weak attacks
        defenders = team_players[team_players['element_type'].isin([1, 2])]  # GK + DEF
        defenders = defenders[defenders['games_played'] >= 3]
        
        if len(defenders) == 0:
            return pd.DataFrame()
            
        # Score based on clean sheet potential and value
        defenders['clean_sheet_rate'] = defenders['season_CS'] / defenders['games_played']
        defenders['matchup_score'] = (
            defenders['clean_sheet_rate'] * 4 +
            defenders['points_per_game'] * 0.6 +
            defenders['points_per_million'] * 0.4
        )
        
        result = defenders.sort_values('matchup_score', ascending=False).head(top_n)
        return result[['web_name', 'position_name', 'season_points', 'now_cost',
                      'clean_sheet_rate', 'points_per_game', 'points_per_million', 
                      'matchup_score']].round(2)

# Find teams with weak defenses (good for attacking players)
weak_defenses = team_rankings.sort_values('defense_rank', ascending=False).head(8)
print("🎯 TEAMS WITH WEAK DEFENSES (Target for Attackers):")
print("=" * 55)
for team in weak_defenses.index:
    defense_rank = int(weak_defenses.loc[team, 'defense_rank'])
    defense_strength = weak_defenses.loc[team, 'defense_strength']
    print(f"{defense_rank:2d}. {team:<15} (Defense: {defense_strength:.3f})")

# Find teams with weak attacks (good for defenders)
weak_attacks = team_rankings.sort_values('attack_rank', ascending=False).head(8)
print(f"\n🛡️ TEAMS WITH WEAK ATTACKS (Good for Defenders):")
print("=" * 50)
for team in weak_attacks.index:
    attack_rank = int(weak_attacks.loc[team, 'attack_rank'])
    attack_strength = weak_attacks.loc[team, 'attack_strength']
    print(f"{attack_rank:2d}. {team:<15} (Attack: {attack_strength:.3f})")

# Example: Best attackers from top teams when facing weak defenses
print(f"\n⚽ ATTACKING PICKS FROM STRONG TEAMS:")
print("=" * 45)
strong_attack_teams = ['Liverpool', 'Man City', 'Arsenal', 'Chelsea']
for team in strong_attack_teams[:2]:  # Show top 2 to save space
    if team in season_stats['team_name'].values:
        attackers = get_players_for_matchup(team, 'weak_defense', season_stats, team_rankings, 3)
        if not attackers.empty:
            print(f"\n🔴 {team} Attackers (vs Weak Defenses):")
            print(attackers[['web_name', 'position_name', 'now_cost', 'goals_per_game', 'assists_per_game', 'points_per_game']].to_string(index=False))


=== SOLUTION 3: PLAYER RECOMMENDATIONS BY OPPONENT STRENGTH ===
Find best players against weak defenses/attacks
🎯 TEAMS WITH WEAK DEFENSES (Target for Attackers):
20. Wolves          (Defense: 0.266)
19. West Ham        (Defense: 0.299)
18. Man Utd         (Defense: 0.308)
17. Burnley         (Defense: 0.331)
16. Nott'm Forest   (Defense: 0.339)
15. Brighton        (Defense: 0.376)
14. Brentford       (Defense: 0.377)
13. Leeds           (Defense: 0.513)

🛡️ TEAMS WITH WEAK ATTACKS (Good for Defenders):
20. Burnley         (Attack: 3.153)
19. Brentford       (Attack: 3.674)
18. Aston Villa     (Attack: 3.721)
17. Wolves          (Attack: 3.726)
16. Fulham          (Attack: 3.915)
15. Newcastle       (Attack: 3.945)
14. West Ham        (Attack: 3.991)
13. Sunderland      (Attack: 4.003)

⚽ ATTACKING PICKS FROM STRONG TEAMS:

🔴 Liverpool Attackers (vs Weak Defenses):
   web_name position_name  now_cost  goals_per_game  assists_per_game  points_per_game
Gravenberch    Midfielder       5.

In [198]:
# ACTIONABLE FIXTURE ANALYSIS SUMMARY
print("\n" + "="*80)
print("=== ACTIONABLE FIXTURE ANALYSIS - HOW TO USE ===")
print("Now you have REAL tools for fixture analysis!")

print(f"\n🔧 WHAT WE'VE BUILT:")
print("1. **Team Strength Rankings** - Know which teams are strong/weak")
print("2. **Fixture Difficulty Calculator** - Score any specific matchup") 
print("3. **Player Recommendations** - Find best players vs weak opponents")

print(f"\n📋 HOW TO USE FOR REAL FPL DECISIONS:")
print("=" * 50)

print(f"\n1️⃣ **CHECK UPCOMING FIXTURES** (From FPL website/app):")
print("   • Go to FPL website fixtures section")
print("   • Note upcoming gameweek fixtures")
print("   • Use our calculator for each matchup")

print(f"\n2️⃣ **USE OUR CALCULATOR**:")
print("   # Example for real fixture analysis")
print("   calculate_fixture_difficulty('Liverpool', 'Brighton', team_rankings, attacking_player=True)")
print("   calculate_fixture_difficulty('Arsenal', 'Burnley', team_rankings, attacking_player=False)")

print(f"\n3️⃣ **IDENTIFY OPPORTUNITIES**:")
print("   🎯 **For Attackers**: Look for fixtures where:")
print("      • Strong attacking teams (Liverpool, Man City, Arsenal)")
print("      • Face weak defenses (Wolves, West Ham, Man Utd, Burnley)")
print("      • Home advantage (+10% boost)")
print("   ")
print("   🛡️ **For Defenders**: Look for fixtures where:")
print("      • Strong defensive teams (Arsenal, Crystal Palace, Newcastle)")
print("      • Face weak attacks (Burnley, Brentford, Aston Villa)")
print("      • Home advantage for clean sheet potential")

print(f"\n4️⃣ **PRACTICAL EXAMPLE PROCESS**:")
print("   If you see 'Arsenal vs Burnley' in upcoming fixtures:")
print("   📊 Arsenal Rank: #2 overall, #1 defense")  
print("   📊 Burnley Rank: #20 attack, #17 defense")
print("   ✅ Arsenal defenders = EXCELLENT pick (strong defense vs weak attack)")
print("   ✅ Arsenal attackers = GREAT pick (good attack vs weak defense)")

print(f"\n💡 **KEY INSIGHTS FROM OUR ANALYSIS**:")
print("   🔥 **Best Attacking Teams**: Liverpool (#1), Man Utd (#2), Man City (#3)")
print("   🛡️ **Best Defensive Teams**: Arsenal (#1), Crystal Palace (#2), Newcastle (#3)")  
print("   🎯 **Weakest Defenses**: Wolves, West Ham, Man Utd, Burnley")
print("   📉 **Weakest Attacks**: Burnley, Brentford, Aston Villa, Wolves")

print(f"\n🚀 **NEXT STEPS**:")
print("   1. Check FPL fixtures for next 3-5 gameweeks")
print("   2. Use our calculator for each matchup")
print("   3. Target players from strong teams vs weak opponents")
print("   4. Consider home/away advantage")
print("   5. Factor in player form and rotation risk")

print(f"\n✅ **THIS IS MUCH BETTER THAN THE ORIGINAL APPROACH**:")
print("   ❌ Before: Fake fixtures (every team vs every team)")
print("   ✅ Now: Real team strength analysis + manual fixture input")
print("   📈 Result: Actionable insights for actual FPL decisions!")


=== ACTIONABLE FIXTURE ANALYSIS - HOW TO USE ===
Now you have REAL tools for fixture analysis!

🔧 WHAT WE'VE BUILT:
1. **Team Strength Rankings** - Know which teams are strong/weak
2. **Fixture Difficulty Calculator** - Score any specific matchup
3. **Player Recommendations** - Find best players vs weak opponents

📋 HOW TO USE FOR REAL FPL DECISIONS:

1️⃣ **CHECK UPCOMING FIXTURES** (From FPL website/app):
   • Go to FPL website fixtures section
   • Note upcoming gameweek fixtures
   • Use our calculator for each matchup

2️⃣ **USE OUR CALCULATOR**:
   # Example for real fixture analysis
   calculate_fixture_difficulty('Liverpool', 'Brighton', team_rankings, attacking_player=True)
   calculate_fixture_difficulty('Arsenal', 'Burnley', team_rankings, attacking_player=False)

3️⃣ **IDENTIFY OPPORTUNITIES**:
   🎯 **For Attackers**: Look for fixtures where:
      • Strong attacking teams (Liverpool, Man City, Arsenal)
      • Face weak defenses (Wolves, West Ham, Man Utd, Burnley)
      •

# 8️⃣ Quick Reference & Usage Guide {#quick-reference--usage-guide}

## 🚀 **How to Use This Notebook for FPL Success**

### 📋 **Quick Start Checklist:**
1. ✅ Run all cells in order (Ctrl+A, then Shift+Enter)
2. ✅ Check the **Team Strength Rankings** (Section 7.2)
3. ✅ Use the **Fixture Difficulty Calculator** for upcoming gameweeks
4. ✅ Review **Top Player Rankings** (Section 6) 
5. ✅ Apply insights to your FPL team

---

## 🎯 **Key Functions You Can Use:**

### 🔧 **Player Analysis Functions:**
```python
# Get top defenders
top_defenders = filter_defenders_corrected(season_stats, min_games=4, top_n=15)

# Get top attackers  
top_attackers = filter_attackers_corrected(season_stats, min_games=4, top_n=15)

# Get only midfielders
midfielders_only = filter_attackers_corrected(season_stats, positions=[3], top_n=10)

# Get only forwards
forwards_only = filter_attackers_corrected(season_stats, positions=[4], top_n=10)
```

### 🏟️ **Fixture Analysis Functions:**
```python
# Calculate difficulty for a specific fixture
difficulty = calculate_fixture_difficulty('Arsenal', 'Burnley', team_rankings, attacking_player=True)

# Get team strength rankings
team_strength = create_team_strength_rankings(season_stats)

# Find players for specific matchups
arsenal_attackers = get_players_for_matchup('Arsenal', 'weak_defense', season_stats, team_rankings)
```

---

## 📊 **Key Data Sources Available:**

| Variable | Description | Use Case |
|----------|-------------|----------|
| `season_stats` | Cumulative season statistics per player | Main analysis dataframe |
| `team_rankings` | Team attacking/defensive strength rankings | Fixture difficulty assessment |
| `top_scorers` | Top 10 season point scorers | Transfer targets |
| `value_players` | Best value players (points per £m) | Budget picks |
| `hidden_gems` | Undervalued players with strong underlying stats | Differential picks |

---

## ⚡ **Real-World FPL Workflow:**

### 🔄 **Weekly Process:**
1. **Check Fixtures** → Go to official FPL website for upcoming gameweek fixtures
2. **Use Our Calculator** → Input each fixture into `calculate_fixture_difficulty()`
3. **Identify Opportunities** → Look for strong teams vs weak opponents
4. **Check Player Form** → Review our player rankings and current form
5. **Make Transfers** → Target players from favorable fixtures
6. **Set Captain** → Choose captain from strongest attacking fixtures

### 🎯 **Example Decision Process:**
```
If Arsenal vs Burnley is upcoming:
✅ Arsenal: #2 overall, #1 defense  
✅ Burnley: #20 attack, #17 defense
→ Arsenal defenders = EXCELLENT pick
→ Arsenal attackers = GOOD pick
```

---

## 📈 **What Makes This Analysis Special:**

### ✅ **Our Advantages:**
- **Accurate Season Data** → Uses cumulative season totals, not misleading gameweek data
- **Expected Stats Integration** → Considers xG, xA, xCS for sustainable performance
- **Value Analysis** → Points per £million calculations for budget optimization
- **Fixture Intelligence** → Real team strength analysis for upcoming matches
- **Actionable Insights** → Ready-to-use recommendations for transfers

### 🎯 **Perfect For:**
- **Transfer Planning** → Identify best players by position and value
- **Captain Selection** → Find players with favorable upcoming fixtures  
- **Budget Optimization** → Discover undervalued players with high potential
- **Fixture Difficulty** → Assess how hard/easy upcoming matches will be
- **Team Strategy** → Long-term planning based on team strength analysis

---

## 💡 **Pro Tips:**

1. **Combine Multiple Factors** → Don't just look at points - consider fixtures, value, and consistency
2. **Check Rotation Risk** → High-value players may be rotated in busy periods
3. **Monitor Form Changes** → Re-run analysis when new gameweek data is available
4. **Budget Balance** → Use value picks to afford premium players in key positions
5. **Captain Strategy** → Always captain players from teams with favorable fixtures

---

## 🔄 **Updating the Analysis:**
When new gameweek data becomes available:
1. Replace `fpl-data-stats.csv` with updated data
2. Re-run the entire notebook (Runtime → Run All)
3. New insights will be automatically generated

**Happy FPL Managing! 🏆**

In [199]:
# 📋 QUICK TEST: Verify Everything Works
print("🧪 TESTING KEY FUNCTIONS...")
print("=" * 50)

try:
    # Test key functions
    test_defenders = filter_defenders_corrected(season_stats, min_games=3, top_n=3)
    test_attackers = filter_attackers_corrected(season_stats, min_games=3, top_n=3) 
    test_fixture = calculate_fixture_difficulty('Arsenal', 'Burnley', team_rankings, attacking_player=True)
    
    print("✅ Defender function: Working")
    print("✅ Attacker function: Working") 
    print("✅ Fixture calculator: Working")
    print("✅ Team rankings: Available")
    print("✅ Season stats: Available")
    
    print(f"\n📊 DATA SUMMARY:")
    print(f"• Total players analyzed: {len(season_stats):,}")
    print(f"• Gameweeks covered: 1-{season_stats['last_gameweek'].max()}")
    print(f"• Teams in analysis: {season_stats['team_name'].nunique()}")
    
    print(f"\n🎯 READY FOR FPL ANALYSIS!")
    print("All functions are working correctly. Scroll up to use the tools!")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please re-run the notebook from the beginning")

🧪 TESTING KEY FUNCTIONS...


✅ Defender function: Working
✅ Attacker function: Working
✅ Fixture calculator: Working
✅ Team rankings: Available
✅ Season stats: Available

📊 DATA SUMMARY:
• Total players analyzed: 758
• Gameweeks covered: 1-6
• Teams in analysis: 20

🎯 READY FOR FPL ANALYSIS!
All functions are working correctly. Scroll up to use the tools!


---

## 🏁 **Notebook Complete!** 

### ✅ **What You Now Have:**
- **Clean, organized analysis** with proper section headers
- **Accurate season statistics** (not misleading gameweek data)
- **Powerful FPL tools** ready for immediate use
- **Clear documentation** with usage examples
- **Real-world applicable** fixture analysis system

### 🎯 **Next Steps:**
1. Bookmark this notebook for weekly FPL analysis
2. Update the CSV file when new gameweeks are released
3. Use the functions to make informed transfer decisions
4. Apply fixture difficulty analysis for captain selection

### 🔗 **Jump to Key Sections:**
- **[Player Rankings](#player-performance-analysis)** - Top performers by position
- **[Team Strength Rankings](#fixture-analysis-system)** - Attack/defense ratings
- **[Fixture Calculator](#fixture-analysis-system)** - Difficulty scoring tool
- **[Usage Guide](#quick-reference--usage-guide)** - How to use everything

**Happy FPL managing! 🏆**